### 1) 데이터 로드 & 전처리

In [2]:
# 전처리는 LSA에서와 동일
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

# 첫번째 훈련용 샘플을 출력
documents[1]

# 20개의 카테고리 종류 출력
print(dataset.target_names)

news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

news_df['clean_doc'][1]

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
# NLTK로부터 불용어 로드
stop_words = stopwords.words('english') 
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) # 불용어 제거

print(tokenized_doc[:5])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object


### 2) 정수 인코딩 & 단어 집합 만들기
* 각 단어에 정수 인코딩
* 각 뉴스에서의 단어 빈도수 기록
* 각 단어를 (word_id, word_frequency)의 형태로
* gensim의 corpora.Dictionary()를 사용

In [3]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [5]:
# 총 학습된 단어 수
len(dictionary)

64281

### 3) LDA 모델 훈련
* 단어 앞의 수치는 단어의 해당 토픽에 대한 기여도
* 인자 passes는 알고리즘의 동작 횟수
* num_words인자로 출력하고 싶은 단어 개수 지정 (default는 10)

In [6]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.014*"printf" + 0.013*"char" + 0.013*"pain" + 0.008*"henrik"')
(1, '0.011*"people" + 0.011*"would" + 0.007*"think" + 0.005*"know"')
(2, '0.009*"people" + 0.007*"government" + 0.006*"state" + 0.006*"would"')
(3, '0.011*"april" + 0.010*"university" + 0.009*"york" + 0.009*"city"')
(4, '0.012*"year" + 0.009*"last" + 0.009*"good" + 0.009*"would"')
(5, '0.017*"period" + 0.011*"pittsburgh" + 0.010*"power" + 0.008*"doug"')
(6, '0.017*"slave" + 0.016*"master" + 0.013*"liar" + 0.011*"easter"')
(7, '0.010*"system" + 0.009*"windows" + 0.009*"drive" + 0.008*"problem"')
(8, '0.012*"exhaust" + 0.009*"adobe" + 0.008*"harley" + 0.007*"chamber"')
(9, '0.010*"encryption" + 0.009*"chip" + 0.009*"system" + 0.008*"keys"')
(10, '0.021*"would" + 0.017*"like" + 0.012*"know" + 0.010*"good"')
(11, '0.027*"file" + 0.016*"program" + 0.012*"window" + 0.012*"output"')
(12, '0.020*"mail" + 0.018*"please" + 0.014*"thanks" + 0.013*"send"')
(13, '0.029*"space" + 0.012*"nasa" + 0.009*"research" + 0.008*"center"')
(

### 4) LDA 시각화
* pyLDAvis 설치 필요
* 모델 출력 결과에서는 번호가 0부터 시작
* 시각호 결과에서는 번호가 1부터 시작함에 유의

In [7]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

### 5) 문서 별 토픽 분포 확인

In [10]:
# 훈련된 LDA모델에 전체 데이터가 정수 인코딩 된 결과를 넣어줘야함
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(1, 0.63138336), (2, 0.20179266), (19, 0.15263997)]
1 번째 문서의 topic 비율은 [(1, 0.58553755), (4, 0.2139364), (7, 0.07271463), (8, 0.0802653), (19, 0.027792793)]
2 번째 문서의 topic 비율은 [(1, 0.23872791), (2, 0.39209944), (10, 0.19468702), (14, 0.016841253), (15, 0.14552121)]
3 번째 문서의 topic 비율은 [(1, 0.21789159), (4, 0.010090458), (5, 0.01607421), (7, 0.10413471), (9, 0.35787547), (10, 0.1934244), (16, 0.03872081), (18, 0.0523797)]
4 번째 문서의 topic 비율은 [(2, 0.049390234), (4, 0.41554067), (11, 0.091769084), (12, 0.058751438), (13, 0.16098447), (17, 0.1976199)]


C:\Users\JIN\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# 데이터 프레임으로 결과 시각화
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

C:\Users\JIN\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

C:\Users\JIN\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,1.0,0.6314,"[(1, 0.63137823), (2, 0.20180193), (19, 0.1526..."
1,1,1.0,0.5857,"[(1, 0.58566433), (4, 0.21368006), (7, 0.07284..."
2,2,2.0,0.3921,"[(1, 0.23875025), (2, 0.39209586), (10, 0.1946..."
3,3,9.0,0.3579,"[(1, 0.21733119), (4, 0.011716662), (5, 0.0160..."
4,4,4.0,0.4155,"[(2, 0.049450785), (4, 0.4154945), (11, 0.0917..."
5,5,1.0,0.5998,"[(0, 0.14039801), (1, 0.5998094), (11, 0.06777..."
6,6,7.0,0.5804,"[(0, 0.01551565), (7, 0.58035934), (10, 0.3151..."
7,7,1.0,0.4276,"[(1, 0.42755336), (2, 0.19411641), (3, 0.02751..."
8,8,3.0,0.3684,"[(1, 0.19223602), (2, 0.21468726), (3, 0.36839..."
9,9,10.0,0.5950,"[(1, 0.087786235), (4, 0.09046056), (7, 0.1257..."
